In [ ]:
import aiida

# must execute it in the first place
aiida.load_profile()


In [ ]:
from aiida.orm import Code
from aiida.orm import Str, Int, Dict, List, Float
from aiida.engine import calcfunction, WorkChain, ToContext, append_
from aiida.plugins import DataFactory, WorkflowFactory
from itertools import cycle
import os

from aiida.engine import calcfunction, workfunction, submit, run
from aiida.engine import run_get_node
from aiida.orm import load_code, load_node
from alamode_aiida.io import load_atoms
from ase import io
from tools import wait_for_node_finished
from alamode_aiida.io import load_atoms_bare

# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
List = DataFactory('list')
TrajectoryData = DataFactory('array.trajectory')


In [ ]:
# codeの定義を行う。
from tools.aiida_support import get_or_create_local_computer, get_or_create_code
from os.path import expanduser
import os


In [ ]:
home = expanduser("~")
work_directory = os.path.join(home, 'aiida')
computer_local = get_or_create_local_computer(work_directory, 'localhost')
print(computer_local)
code = get_or_create_code('alamode.displace_pf',
                          computer_local,
                          '/home/max/Documents/alamode/displace.py')
code_displace_pf = code
code = get_or_create_code('alamode.displace_random',
                          computer_local,
                          '/home/max/Documents/alamode/displace.py')
code_displace_random = code
code = get_or_create_code('alamode.lammps',
                          computer_local,
                          '/usr/bin/lammps')

code_lammps_name = Str('alamode.lammps@localhost')

code = get_or_create_code('alamode.extract',
                          computer_local,
                          '/home/max/Documents/alamode/tools/extract.py')
code_extract = code
code_extract_name = Str('alamode.extract@localhost')


code_displace_random, code_displace_pf, code_lammps_name


In [ ]:
import json
with open("setting.json") as f:
    _dic = json.load(f)
print(_dic)
CWD = _dic["CWD"]
_path = os.path.abspath(CWD)
os.makedirs(_path, exist_ok=True)
_path


In [ ]:

from os.path import expanduser
from tools import NodeBank


# 再実行時に作成したノードを用いるためにnodebankを使う。
g_force = False
nodebank = NodeBank(_path, force=g_force)


In [ ]:
g_prefix = nodebank.load('cubic_prefix')
g_norder = nodebank.load('cubic_norder')
g_prefix, g_norder


In [ ]:
g_cwd = nodebank.load(f"{g_prefix.value}_cwd")  # ファイル保存directory
g_superstructure = nodebank.load("superstructure")  # 使用する長周期構造StructureData

g_file_format = nodebank.load("format")  # lammpsの位置フォーマット
g_file_format, g_cwd


## for LAMMPS

In [ ]:
key = "format"
g_format = nodebank.load_or_dump(key, Str("LAMMPS"))  # alamodeのファイルオプションの定義


In [ ]:
from aiida_lammps.tests.utils import (
    get_or_create_local_computer, get_or_create_code)
from aiida_lammps.tests.utils import lammps_version


In [ ]:
computer_local = get_or_create_local_computer('work_directory', 'localhost')
code_lammps_force = get_or_create_code('lammps.force', computer_local, 'lammps')


In [ ]:
code_lammps_force_name = Str(code_lammps_force.label+"@"+'localhost')



In [ ]:
g_mag = nodebank.load_or_dump(f"{g_prefix.value}_mag", Float(0.04))


In [ ]:

displacement_patterns = nodebank.load(f'{g_prefix.value}_pattern')
displacement_patterns[1]


In [ ]:
g_action = "random"

if g_action == "pf":

    #codename = "alamode.displace_pf@tutor"
    #code = Code.get_from_string(codename)
    code = code_displace_pf

    builder = code.get_builder()
    builder.format = g_format
    builder.cwd = Str(os.path.join(g_cwd.value, f"{g_prefix.value}_suggest_pf"))
    builder.prefix = g_prefix
    builder.structure_org = g_superstructure
    builder.mag = g_mag
    builder.pattern = displacement_patterns

    g_displacefuture = nodebank.load_code_or_wait_for_node(f"{g_prefix.value}_displace_pf", builder)


elif g_action == "random":

    g_num_disp = nodebank.dump(f"{g_prefix.value}_num_disp", Int(40))

    code = code_displace_random

    builder = code.get_builder()
    builder.format = g_format
    builder.cwd = Str(os.path.join(g_cwd.value, f"{g_prefix.value}_suggest_random"))
    builder.prefix = g_prefix
    builder.structure_org = g_superstructure
    builder.mag = g_mag
    builder.num_disp = g_num_disp
    

    g_displacefuture = nodebank.load_code_or_wait_for_node(f"{g_prefix.value}_suggest_random", builder)

else:
    raise ValueError("unknown action")


In [ ]:
g_displacefuture.outputs.retrieved.list_object_names()


In [ ]:
g_displaced_structures = g_displacefuture.outputs.displaced_structures
g_displaced_structures


# load tersoff potential

In [ ]:
import json
potentialfile = "lammps_input/potentials/tersoff.json"

with open(potentialfile, "r") as f:
    potential_dict = json.load(f)
lammps_potential = DataFactory("lammps.potential")(
    type=potential_dict["pair_style"], data=potential_dict["potential_dict"]
)
lammps_potential


In [ ]:
code_lammps_force_name


# execution option

In [ ]:
lammps_parameters = DataFactory('dict')(dict={
    'lammps_version': lammps_version(),
    'output_variables': ["temp", "etotal", "pe", "ke"],
    'thermo_keywords': []
})
meta_options = {
    "resources": {
        "num_machines": 1,
        "num_mpiprocs_per_machine": 1}
}


# run all the displacements

In [ ]:
Code.get_from_string(code_lammps_force_name.value)


In [ ]:
alldisp_lammps_WorkChain = WorkflowFactory("alamode.force_simulator_lammps")


In [ ]:
g_prefix


In [ ]:
inputs = {"structures": g_displaced_structures,
          "parameters": lammps_parameters,
          "code_string":  code_lammps_force_name,
          #"cwd": Str(os.path.join(g_cwd.value, "cubic_all_run")),
          "prefix": g_prefix,
          'potential': lammps_potential}



In [ ]:
# It will take 30 min. for GaN222.


In [ ]:
from datetime import datetime
starttime = datetime.now()

lammps_all = nodebank.load(f"{g_prefix.value}_all_lammps", raise_error=False)
if lammps_all is None:
    lammps_all = submit(alldisp_lammps_WorkChain, **inputs)
    print(lammps_all)
    wait_for_node_finished(lammps_all, 5)
    print(lammps_all.is_finished_ok)
    if lammps_all.is_finished_ok:
        nodebank.dump(f"{g_prefix.value}_all_lammps", lammps_all)
endtime = datetime.now()
print(endtime-starttime)


In [ ]:
if False:
    # too big to show
    pk = lammps_all.pk
    print(pk)
    !verdi node graph generate $pk
    from wand.image import Image as WImage
    filename = f"{pk}.dot.pdf"
    img = WImage(filename=filename)
    display(img)

# extract

In [ ]:
g_target_dict = lammps_all.outputs.displacement_and_forces

In [ ]:
g_superstructure


In [ ]:
code_extract = Code.get_from_string(code_extract_name.value)
code_extract


In [ ]:
builder = code_extract.get_builder()
builder.format = g_format
builder.structure_org = g_superstructure
# builder.cwd = Str(os.path.join(g_cwd.value, f"{g_prefix.value}_extract"))
builder.prefix = g_prefix
builder.displacement_and_forces = g_target_dict
# builder.parameters = Dict(dict={"LAMMPS":{"lammps-style":"atomic"}})


In [ ]:
g_extract = submit(builder)
wait_for_node_finished(g_extract)


In [ ]:
nodebank.dump(f"{g_prefix.value}_extract", g_extract)


In [ ]:
print("done")


In [ ]:
g_graph=True
if g_graph:
    
    pk = g_extract.pk
    print(pk)
    !verdi node graph generate $pk
    from wand.image import Image as WImage
    filename = f"{pk}.dot.pdf"
    img = WImage(filename=filename)
    display(img)
